In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum,avg,col,min,max,upper, lower, trim, ltrim, rtrim, substring 
from pyspark.sql.functions import substring_index, split, repeat, rpad, lpad, regexp_replace, regexp_extract, length, instr, initcap
spark=SparkSession.builder.appName("func").getOrCreate()
# data= [
#     (101, "Alice", "HR", 40000, 29),
#     (102, "Bob", "IT", 55000, 34),
#     (103, "Charlie", "Finance", 48000, 28),
#     (104, "David", "IT", 61000, 41),
#     (105, "Eve", "HR", 39000, 25),
#     (106, "Frank", "Finance", 53000, 38)
]

df=spark.createDataFrame(data,["id","name","dept","sal","age"])

df.show()
df.select("*").show()
df.select("id","dept").show()
df.where(col("dept") =="IT").show()
df.filter(col("age")>29).show()
df.filter(col("sal").between(30000, 40000)).show()
df.groupBy("dept").agg(avg(col("sal").alias("dept_avg_sal"))).show()
print(df.columns)
spark.stop() 

In [0]:
from pyspark.sql.functions import col, rand
from pyspark.storagelevel import StorageLevel
import time

num_records = 5000000
df = spark.range(1, num_records + 1) \
    .withColumn("region", (col("id") % 10)) \
    .withColumn("category", (col("id") % 5)) \
    .withColumn("sales", (rand(seed=42) * 1000).cast("double"))

display(df.limit(10))
print(f" Total Records: {df.count()}")


print(" Running query without caching...")

start = time.time()
df.filter(col("region") == 3).groupBy("category").avg("sales").collect()
end = time.time()

print(f" Without cache: {end - start:.2f} seconds")


print(" Caching DataFrame in memory...")
df.cache()
df.count()  



start = time.time()
df.filter(col("region") == 3).groupBy("category").avg("sales").collect()
end = time.time()

print(f" With cache: {end - start:.2f} seconds")


print(" Persisting with MEMORY_AND_DISK storage level...")
df.unpersist()
df.persist(StorageLevel.MEMORY_AND_DISK)
df.count()

start = time.time()
df.filter(col("region") == 3).groupBy("category").avg("sales").collect()
end = time.time()

print(f" With persist (MEMORY_AND_DISK): {end - start:.2f} seconds")

df.unpersist()
print(" Cache cleared. Demo complete!")
